In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, Aer, execute
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import depolarizing_error, thermal_relaxation_error

def create_noise_model(p_depol, T1, T2):
    noise_model = NoiseModel()
    dep_error = depolarizing_error(p_depol, 1)
    therm_error = thermal_relaxation_error(T1, T2, p_depol)
    noise_model.add_all_qubit_quantum_error(dep_error, ['u1', 'u2', 'u3'])
    noise_model.add_all_qubit_quantum_error(therm_error, ['u1', 'u2', 'u3'])
    return noise_model

def quantum_circuit(num_measurements, rotation_angle):
    qc = QuantumCircuit(1, num_measurements)
    qc.h(0)  # Put qubit in superposition
    for i in range(num_measurements):
        qc.measure(0, i)
        qc.ry(rotation_angle, 0)  # Apply rotation after measurement
    return qc

def run_quantum_experiment(num_measurements, num_shots, p_depol, T1, T2, rotation_angle):
    qc = quantum_circuit(num_measurements, rotation_angle)
    noise_model = create_noise_model(p_depol, T1, T2)
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=num_shots, noise_model=noise_model)
    result = job.result().get_counts()
    return result

def classical_simulation(num_measurements, num_shots, p_error):
    results = []
    for _ in range(num_shots):
        measurement = np.random.choice([0, 1])  # Initial random bit
        sequence = [measurement]
        for _ in range(1, num_measurements):
            if np.random.random() < p_error:
                measurement = 1 - measurement  # Flip the bit with probability p_error
            sequence.append(measurement)
        results.append(''.join(map(str, sequence)))
    return {result: results.count(result) for result in set(results)}

def analyze_results(result, num_measurements):
    total_shots = sum(result.values())
    unique_results = len(result)
    entropy = sum(-count/total_shots * np.log2(count/total_shots) for count in result.values())
    return unique_results, entropy

def compare_quantum_classical(max_measurements, num_shots, p_depol, T1, T2, rotation_angle):
    quantum_unique = []
    quantum_entropy = []
    classical_unique = []
    classical_entropy = []

    for num_measurements in range(1, max_measurements + 1):
        # Quantum experiment
        q_result = run_quantum_experiment(num_measurements, num_shots, p_depol, T1, T2, rotation_angle)
        q_unique, q_entropy = analyze_results(q_result, num_measurements)
        quantum_unique.append(q_unique)
        quantum_entropy.append(q_entropy)

        # Classical simulation
        c_result = classical_simulation(num_measurements, num_shots, p_depol)
        c_unique, c_entropy = analyze_results(c_result, num_measurements)
        classical_unique.append(c_unique)
        classical_entropy.append(c_entropy)

    return quantum_unique, quantum_entropy, classical_unique, classical_entropy

# Experiment parameters
max_measurements = 10
num_shots = 1000
p_depol = 0.1
T1, T2 = 50, 30  # Relaxation and dephasing times (in arbitrary units)
rotation_angle = np.pi/4

# Run comparison
q_unique, q_entropy, c_unique, c_entropy = compare_quantum_classical(
    max_measurements, num_shots, p_depol, T1, T2, rotation_angle)

# Plotting
plt.figure(figsize=(12, 5))

plt.subplot(121)
plt.plot(range(1, max_measurements + 1), q_unique, 'b-', label='Quantum')
plt.plot(range(1, max_measurements + 1), c_unique, 'r--', label='Classical')
plt.xlabel('Number of Measurements')
plt.ylabel('Number of Unique Results')
plt.title('Unique Results: Quantum vs Classical')
plt.legend()

plt.subplot(122)
plt.plot(range(1, max_measurements + 1), q_entropy, 'b-', label='Quantum')
plt.plot(range(1, max_measurements + 1), c_entropy, 'r--', label='Classical')
plt.xlabel('Number of Measurements')
plt.ylabel('Entropy of Results')
plt.title('Entropy: Quantum vs Classical')
plt.legend()

plt.tight_layout()
plt.show()

# Calculate and print efficiency gain
unique_gain = [(q - c) / c * 100 for q, c in zip(q_unique, c_unique)]
entropy_gain = [(q - c) / c * 100 for q, c in zip(q_entropy, c_entropy)]

print("Efficiency gain in unique results (%): ")
print(", ".join(f"{gain:.2f}" for gain in unique_gain))

print("\nEfficiency gain in entropy (%): ")
print(", ".join(f"{gain:.2f}" for gain in entropy_gain))